## Importing RAW tiles

In [1]:
# Mounting Drive to Colab to access data
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
inputFolder = '/content/drive/MyDrive/Upsampling/select/gaza_BaA_2'

## Real-ESRGAN (From guidance on GitHUB README)

1. Getting models and environment/dependencies from ERSGAN GitHub

In [ ]:
# Cloninh Real-ESRGAN
!git clone https://github.com/xinntao/Real-ESRGAN.git
%cd Real-ESRGAN
# Setting up environment
!pip install basicsr
!pip install facexlib
!pip install gfpgan
!pip install -r requirements.txt
!python setup.py develop
%ls

Cloning into 'Real-ESRGAN'...
remote: Enumerating objects: 759, done.
remote: Counting objects: 100% (121/121), done.
remote: Compressing objects: 100% (23/23), done.
remote: Total 759 (delta 106), reused 98 (delta 98), pack-reused 638 (from 1)
Receiving objects: 100% (759/759), 5.38 MiB | 41.10 MiB/s, done.
Resolving deltas: 100% (415/415), done.
/content/Real-ESRGAN
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 172.5/172.5 kB 5.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.8/46.8 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 299.4/299.4 kB 15.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 94.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 256.2/256.2 kB 27.6 MB/s eta 0:00:00
  Created wheel for basicsr: filename=basicsr-1.4.2-py3-none-any.whl size=214817 sha256=ba8afc5fa630255695ef94afcfbc6713e06de4e31ddf89e31d16fad80f88163e
  Stored in directory: /root/.cache

2. Fixing import error that occured during inference

In [ ]:
with open('/usr/local/lib/python3.12/dist-packages/basicsr/data/degradations.py', 'r') as file:
    fileData = file.read()

fileData = fileData.replace('from torchvision.transforms.functional_tensor import rgb_to_grayscale',
                               'from torchvision.transforms.functional import rgb_to_grayscale')

with open('/usr/local/lib/python3.12/dist-packages/basicsr/data/degradations.py', 'w') as file:
    file.write(fileData)

3. Moving Files to ERSGAN Inference Path

In [ ]:
import os
from google.colab import files
import shutil

uploadFolder = 'upload'
resultFolder = 'results'

if os.path.isdir(uploadFolder):
    shutil.rmtree(uploadFolder)
if os.path.isdir(resultFolder):
    shutil.rmtree(resultFolder)
os.mkdir(uploadFolder)
os.mkdir(resultFolder)

# # manually upload images
# uploaded = files.upload()
# for filename in uploaded.keys():
#   dst_path = os.path.join(uploadFolder, filename)
#   print(f'move {filename} to {dst_path}')
#   shutil.move(filename, dst_path)


# Read images from the specified path and move them to the upload folder
if os.path.isdir(inputFolder):
    for filename in os.listdir(inputFolder):
        src_path = os.path.join(inputFolder, filename)
        if os.path.isfile(src_path):
            dstPath = os.path.join(uploadFolder, filename)
            print(f'copy {filename} to {dstPath}')
            shutil.copy(src_path, dstPath)
else:
    print(f"Error: Input path '{inputFolder}' not found.")

copy Copy of 2024-10-20.png to upload/Copy of 2024-10-20.png
copy Copy of 2025-03-02.png to upload/Copy of 2025-03-02.png
copy Copy of 2024-06-02.png to upload/Copy of 2024-06-02.png


4. Run inference on the image tiles

In [ ]:
# if it is out of memory, try to use the `--tile` option
# We upsample the image with the scale factor X3.5
!python inference_realesrgan.py -n RealESRGAN_x4plus -i upload --outscale 3.5 --face_enhance
# Arguments
# -n, --model_name: Model names
# -i, --input: input folder or image
# --outscale: Output scale, can be arbitrary scale factore.

Downloading: "https://github.com/xinntao/Real-ESRGAN/releases/download/v0.1.0/RealESRGAN_x4plus.pth" to /content/Real-ESRGAN/weights/RealESRGAN_x4plus.pth

100% 63.9M/63.9M [00:00<00:00, 288MB/s]
/usr/local/lib/python3.12/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)
Downloading: "https://github.com/xinntao/facexlib/releases/download/v0.1.0/detection_Resnet50_Final.pth" to /content/Real-ESRGAN/gfpgan/weights/detection_Resnet50_Final.pth

100% 104M/104M [00:00<00:00, 324MB/s] 
Downloading: "https://github.com/xinntao/facexlib/releases/download/v0.2.2/parsing

5. Visualise Outputs

In [ ]:
# utils for visualisation
import cv2
import matplotlib.pyplot as plt

def display(img1, img2):
  fig = plt.figure(figsize=(25, 10))
  ax1 = fig.add_subplot(1, 2, 1)
  plt.title('Input image', fontsize=16)
  ax1.axis('off')
  ax2 = fig.add_subplot(1, 2, 2)
  plt.title('Real-ESRGAN output', fontsize=16)
  ax2.axis('off')
  ax1.imshow(img1)
  ax2.imshow(img2)

def imread(imgPath):
  img = cv2.imread(imgPath)
  img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
  return img

# display each image in the upload folder
import os
import glob

inputFolder = 'upload'
resultFolder = 'results'
inputLst = sorted(glob.glob(os.path.join(inputFolder, '*')))
outputLst = sorted(glob.glob(os.path.join(resultFolder, '*')))
for inputPath, outputPath in zip(inputLst, outputLst):
  imgInput = imread(inputPath)
  imgOutput = imread(outputPath)
  display(imgInput, imgOutput)

6. Extract Outputs in Folder

EXTERNAL DWNLOAD

In [ ]:
zipFilename = 'Real-ESRGAN_result.zip'
if os.path.exists(zipFilename):
  os.remove(zipFilename)
os.system(f"zip -r -j {zipFilename} results/*")
files.download(zipFilename)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>